In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import tensorflow as tf
from helper_functions import *
importTensorflow(memory=4090)
import numpy as np
import time

2.16.1
1 Physical GPUs, 1 Logical GPUs


In [2]:
if "input.txt" not in os.listdir():
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# character level tokenizer
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
data = tf.constant(encode(text), dtype=tf.int64)
print(data.shape, data.dtype)
print(data[:100])

(1115394,) <dtype: 'int64'>
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)


In [8]:
# split data
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1].numpy()

array([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is "{context}", the target: {target}')

when input is "[18]", the target: 47
when input is "[18 47]", the target: 56
when input is "[18 47 56]", the target: 57
when input is "[18 47 56 57]", the target: 58
when input is "[18 47 56 57 58]", the target: 1
when input is "[18 47 56 57 58  1]", the target: 15
when input is "[18 47 56 57 58  1 15]", the target: 47
when input is "[18 47 56 57 58  1 15 47]", the target: 58


In [11]:
tf.random.set_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = np.random.randint(low = 0, high = len(data) - block_size, size=(batch_size, ))
    x = tf.stack([data[i:i+block_size] for i in ix])
    y = tf.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print("-"*20)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(f'when input is "{context}", the target: {target}')

inputs:
(4, 8)
tf.Tensor(
[[46 39 58  1 58 46 43 63]
 [ 1 54 39 57 58  1 50 47]
 [ 1 46 43  5 57  1 56 43]
 [53 57 43  6  0 37 53 59]], shape=(4, 8), dtype=int64)
targets:
(4, 8)
tf.Tensor(
[[39 58  1 58 46 43 63  1]
 [54 39 57 58  1 50 47 44]
 [46 43  5 57  1 56 43 58]
 [57 43  6  0 37 53 59 56]], shape=(4, 8), dtype=int64)
--------------------
when input is "[46]", the target: 39
when input is "[46 39]", the target: 58
when input is "[46 39 58]", the target: 1
when input is "[46 39 58  1]", the target: 58
when input is "[46 39 58  1 58]", the target: 46
when input is "[46 39 58  1 58 46]", the target: 43
when input is "[46 39 58  1 58 46 43]", the target: 63
when input is "[46 39 58  1 58 46 43 63]", the target: 1
when input is "[1]", the target: 54
when input is "[ 1 54]", the target: 39
when input is "[ 1 54 39]", the target: 57
when input is "[ 1 54 39 57]", the target: 58
when input is "[ 1 54 39 57 58]", the target: 1
when input is "[ 1 54 39 57 58  1]", the target: 50
when inpu

In [12]:
# Stack example

ix = np.random.randint(low = 0, high = len(data) - block_size, size=(4, ))
x = tf.stack([data[i:i+block_size] for i in ix])
x

<tf.Tensor: shape=(4, 8), dtype=int64, numpy=
array([[ 1, 54, 53, 61, 43, 56,  1, 51],
       [52, 43, 61, 57,  1, 53, 44,  1],
       [56,  1, 58, 46, 43,  1, 47, 52],
       [53,  1, 58, 46, 43,  1, 57, 43]])>

In [13]:
print(xb)

tf.Tensor(
[[46 39 58  1 58 46 43 63]
 [ 1 54 39 57 58  1 50 47]
 [ 1 46 43  5 57  1 56 43]
 [53 57 43  6  0 37 53 59]], shape=(4, 8), dtype=int64)


In [14]:
tf.random.set_seed(1337)

class BigramLanguageModel(tf.keras.Model):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, vocab_size)
    
    def call(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        loss = None
        if targets is not None:
            lossF = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            # print(idx.shape, targets.shape, logits.shape)
            loss = lossF(targets, logits)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # print(logits.shape)
            # print(logits)
            logits = logits[:, -1, :]
            probs = tf.nn.softmax(logits, axis=-1)
            idx_next = tf.random.categorical(probs, num_samples = 1)
            idx = tf.concat([idx, idx_next], axis=-1)
            # print(_, idx)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = tf.zeros((1,1), dtype=tf.int64)
print(decode(m.generate(idx, max_new_tokens=100)[0].numpy()))

(4, 8, 65)
tf.Tensor(4.1714725, shape=(), dtype=float32)

UZiQlH
XYbrOU&py,dn-ovbsESOAqBKsk:yxV'-lNES!OU$3?xB,CpHqR

y;TFC,MjPQ3SbQ,SjCJ.hetJ.Mv!WQ?q,&A.M KHB


In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [16]:
print(m.trainable_variables)

[<KerasVariable shape=(65, 65), dtype=float32, path=bigram_language_model/embedding/embeddings>]


In [17]:
@tf.function
def train_step(xb, yb):
    with tf.GradientTape() as tape:
        logits, loss = m(xb, yb)
    gradients = tape.gradient(loss, m.trainable_variables)
    optimizer.apply_gradients(zip(gradients, m.trainable_variables))
    return loss

In [18]:
batch_size = 32

for steps in range(1000):
    xb, yb = get_batch('train')
    loss = train_step(xb, yb)
print("Loss :",loss.numpy())


Loss : 2.8689694


In [19]:
print(decode(m.generate(idx, max_new_tokens=500)[0].numpy()))


OfrXJkURRD!zvrG&onkf!
Sx3;VgugnVa
ZZ;zy&fJYPHUsg$pq:SpKDArjSlKgsyFYefclQs?MLBRYAT;HTO,V?c'P?kE$rrsebWWMcKmmP:AXlTYVLJe!?dH
lVGpp OSzQdfx&w3LzxJ
Oh;ySAtvZo,IY

JuCTL:,ZOQYWQuP
sDRps$hBC MmLnE3u!rleAIGXq:yd
N.yo-pov.CXBX;fZIr'GHM$,Th:33ru;-fGis'?X mmHSRdUdVtMjLad lkz!jW'JPUiyGezC,3z$T,-r.qpa;
UFRy;dcad$RMicrVqbUJ,nM3ZJRYi&b;,ALM3He;''!C-MgWv;d
Qz:sXidPI,;yrr?gu,TSPqFA'aTSBTs$yz&Mn hTK&uQDTrPQxVFem iz ttEKGteVedvKj,AeL'Le w&p!RLztkd;iC.KfSkYe?sB.OF$VOHif-:a'?hCtozVwk,yUOXBbHQMGinGv!3rGBwhIEFVQ,eHNS


In [20]:
def estimate_loss(model, eval_iters, get_batch):
    results = {}
    for split in ['train', 'val']:
        losses = []
        for _ in range(eval_iters):
            xb, yb = get_batch(split)
            _, loss = model(xb, yb)
            losses.append(loss.numpy())
        results[split] = tf.reduce_mean(losses)
    return results

In [21]:
max_iters = 1000
eval_interval = 100
eval_iters = 10

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss(m, eval_iters, get_batch)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Perform a training step
    loss = train_step(xb, yb)

    # Print loss for each step (optional)
    print(f"Step {iter}, Loss: {loss.numpy()}")

    # Generate from the model every `eval_interval` iterations
    if iter % eval_interval == 0:
        context = tf.zeros((1, 1), dtype=tf.int64)
        generated_sequence = m.generate(context, max_new_tokens=500)
        print(decode(generated_sequence[0].numpy()))

step 0: train loss 2.8992, val loss 2.9129
Step 0, Loss: 2.855123519897461

rS:A.DBAoB$'ASG;;r SR3!H!NlF?gcoIheO; O-fEa tNTHG,Z!NKMH&YfExEBawBg WhG-F&Bj
qlzfbcYGEhy:gguDuaT
,lrcAeCkRgK;jhtMY-wxYLfZKF QJpnyx,Ayccc3ziKNG,LusaA?UTENTgS'xDFi-LtA;ujVb;Mx'TWEcCVyXrp$JB-,VdQ
BIASpwDXVNpQyHJdxSka
,eTigQbDHooKIyQ3wfJR,DlGobLY,d;b;&
dO:xDmdWu;eLSWSEGx;V&X'ioJW!,yH:PJBRK,ZizhX;E.Ul!nIAO,,$3:
nzTDS;WSQPqez?ErXTO&&?A$&XNz;eil?EFQSuahbndyU-Zh,RDN&bWIiv:rpBafjCQJZ'oKBRh,bCo!vKfIib.Law&uvmfnozHmcYVCG&Q UBoHuN
rPDuCyso!U&V;pSzfZylVf.LeEVQbCBgESuxrY;REeY$L,?CLNG3kVtFkFMgB pplQ&IfSZd:Ama'
Step 1, Loss: 2.7479662895202637
Step 2, Loss: 2.9343531131744385
Step 3, Loss: 2.839909553527832
Step 4, Loss: 2.8764820098876953
Step 5, Loss: 2.8657326698303223
Step 6, Loss: 2.896409511566162
Step 7, Loss: 2.92138409614563
Step 8, Loss: 2.9125213623046875
Step 9, Loss: 2.9104158878326416
Step 10, Loss: 2.7994558811187744
Step 11, Loss: 2.826650857925415
Step 12, Loss: 2.7822105884552
Step 13, Loss: 2.811686992645263

# Self attention

In [22]:
tf.random.set_seed(1337)
B,T,C = 4,8,2
x = tf.random.normal((B,T,C))
print(x.shape)
print(x.numpy())

(4, 8, 2)
[[[-3.4665978e-01 -1.1499242e+00]
  [-6.0309816e-02  6.9729918e-01]
  [ 1.6482359e-01  8.2753563e-01]
  [-1.2660017e+00 -7.5791830e-01]
  [ 1.3696648e-01  1.2408369e+00]
  [ 3.0636016e-01 -9.6362972e-01]
  [-1.6146293e+00  2.2820495e-03]
  [ 1.9042031e-01 -1.1532472e+00]]

 [[ 6.4064187e-01  1.0989506e+00]
  [-6.6641438e-01 -2.2210772e+00]
  [-1.5676336e-01  1.0218153e+00]
  [-2.3248911e-01  1.6681559e-01]
  [ 1.5252142e+00  7.0989805e-01]
  [-5.8470812e-02 -1.0795627e+00]
  [-4.7704136e-01 -6.9138509e-01]
  [-4.1711381e-01 -6.1739590e-03]]

 [[-1.4833307e-03  2.4637158e+00]
  [ 1.0176952e+00  1.4598781e-01]
  [-1.3405284e+00  4.2292166e-01]
  [ 2.4836320e-01 -1.5479729e+00]
  [ 7.5895917e-01  7.3927677e-01]
  [-1.8256147e+00 -6.4090222e-01]
  [-1.0602007e-01  1.9212660e+00]
  [-3.1409055e-01  1.8654646e-01]]

 [[ 1.8926893e+00 -1.8999275e+00]
  [ 4.9457046e-01 -1.2837032e+00]
  [-1.2634312e+00 -8.0137241e-01]
  [-2.3291025e-01 -1.0270079e+00]
  [-2.0107047e-01  6.8568058e-02

In [23]:
xbow = tf.Variable(tf.zeros((B,T,C)))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t].assign(tf.reduce_mean(xprev, axis=0))

In [24]:
x[0]

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[-0.34665978, -1.1499242 ],
       [-0.06030982,  0.6972992 ],
       [ 0.16482359,  0.8275356 ],
       [-1.2660017 , -0.7579183 ],
       [ 0.13696648,  1.2408369 ],
       [ 0.30636016, -0.9636297 ],
       [-1.6146293 ,  0.00228205],
       [ 0.19042031, -1.1532472 ]], dtype=float32)>

In [25]:
xbow[0]

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[-0.34665978, -1.1499242 ],
       [-0.2034848 , -0.22631249],
       [-0.08071534,  0.12497022],
       [-0.37703693, -0.09575191],
       [-0.27423626,  0.17156585],
       [-0.17747019, -0.01763342],
       [-0.38277864, -0.01478835],
       [-0.31112877, -0.15709572]], dtype=float32)>

In [26]:
np.tril(tf.ones((3,3)))

array([[1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.]], dtype=float32)

Exploring a faster way to calculate average of token in time steps instead of forloops, use lower traingular matrix multiplication to get a quick sum

In [27]:
tf.random.set_seed(42)
a = tf.constant(np.tril(tf.ones((3,3), dtype=tf.float32)))
a = a / tf.reduce_sum(a, axis=1, keepdims=True)
b = tf.constant(np.random.randint(0, 10, (3,2)), dtype=tf.float32)
print("a=")
print(a.numpy())
print("b=")
print(b.numpy())
print("c=")
c = a @ b
print(c.numpy())

a=
[[1.         0.         0.        ]
 [0.5        0.5        0.        ]
 [0.33333334 0.33333334 0.33333334]]
b=
[[8. 6.]
 [5. 8.]
 [1. 8.]]
c=
[[8.       6.      ]
 [6.5      7.      ]
 [4.666667 7.333334]]


In [28]:
# doing actually on array
wei = tf.constant(np.tril(tf.ones((T,T), dtype=tf.float32)))
wei = wei / tf.reduce_sum(wei, axis=1, keepdims=True)
xbow2 = wei @ x
xbow2[0]

<tf.Tensor: shape=(8, 2), dtype=float32, numpy=
array([[-0.34665978, -1.1499242 ],
       [-0.2034848 , -0.22631249],
       [-0.08071534,  0.12497024],
       [-0.37703693, -0.09575191],
       [-0.27423626,  0.17156585],
       [-0.1774702 , -0.0176334 ],
       [-0.38277864, -0.01478835],
       [-0.31112874, -0.15709572]], dtype=float32)>

masked fill

In [29]:
tril = tf.constant(np.tril(tf.ones((T,T), dtype=tf.float32)))
wei = tf.zeros((T,T))
wei = tf.where(tril == 0, x = float('-inf'), y = 0)
print(wei)
wei = tf.nn.softmax(wei)
xbow3 = wei @ x
print(xbow3[0])

tf.Tensor(
[[  0. -inf -inf -inf -inf -inf -inf -inf]
 [  0.   0. -inf -inf -inf -inf -inf -inf]
 [  0.   0.   0. -inf -inf -inf -inf -inf]
 [  0.   0.   0.   0. -inf -inf -inf -inf]
 [  0.   0.   0.   0.   0. -inf -inf -inf]
 [  0.   0.   0.   0.   0.   0. -inf -inf]
 [  0.   0.   0.   0.   0.   0.   0. -inf]
 [  0.   0.   0.   0.   0.   0.   0.   0.]], shape=(8, 8), dtype=float32)
tf.Tensor(
[[-0.34665978 -1.1499242 ]
 [-0.2034848  -0.22631249]
 [-0.08071534  0.12497024]
 [-0.37703693 -0.09575191]
 [-0.27423626  0.17156585]
 [-0.1774702  -0.0176334 ]
 [-0.38277864 -0.01478835]
 [-0.31112874 -0.15709572]], shape=(8, 2), dtype=float32)


In [30]:
#version 4: self attention
tf.random.set_seed(1337)
B,T,C = 4,8,32
x = tf.random.normal((B,T,C))

tril = tf.constant(np.tril(tf.ones((T,T), dtype=tf.float32)))
wei = tf.zeros((T,T))
wei = tf.where(tril==0, x = float('-inf'), y = 0)
wei = tf.nn.softmax(wei)
out = wei @ x
out.shape

TensorShape([4, 8, 32])

In [33]:
# single head self attention
head_size = 16

key = tf.keras.layers.Dense(head_size, use_bias=False)
query = tf.keras.layers.Dense(head_size, use_bias=False)
value = tf.keras.layers.Dense(head_size, use_bias=False)
k = key(x)
q = query(x)
wei = q @ tf.transpose(k, perm=[0, 2, 1]) # (B, T, 16) @ (B, 16, T) -> (B, T, T)


tril = tf.constant(np.tril(tf.ones((T,T), dtype=tf.float32)))
wei = tf.where(tril == 0, x = float('-inf'), y=wei)
wei = tf.nn.softmax(wei)

v = value(x)
out = wei @ v
# out = wei @ x

print(out.shape)

(4, 8, 16)


In [34]:
wei[0]

<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [4.5326275e-01, 5.4673725e-01, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [9.6316671e-01, 1.6011413e-02, 2.0821916e-02, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [8.7811244e-01, 7.6494224e-02, 3.5347056e-02, 1.0046349e-02,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.4197586e-02, 4.8992136e-01, 4.4702160e-01, 4.3273874e-02,
        5.5856062e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.3771570e-05, 1.9450166e-03, 1.0062826e-02, 9.7485387e-01,
        2.1260006e-03, 1.0998492e-02, 0.0000000e+00, 0.0000000e+00],
       [1.6136803e-05, 3.3624191e-04, 6.4117964e-03, 1.6444256e-04,
        9.9220520e-01, 8.4988237e-04, 1.6279737e-05, 0.0000000

In [43]:
# scaled dot product attention
wei = q @ tf.transpose(k, perm=[0, 2, 1])
tf.math.reduce_variance(wei).numpy()

21.962606

In [44]:
wei = q @ tf.transpose(k, perm=[0, 2, 1]) * head_size**(-0.5)
tf.math.reduce_variance(wei).numpy()

1.3726629